In [2]:
import numpy as np
import pandas as pd
import sys
!{sys.executable} -m pip install imbalanced-learn
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier

In [3]:
import google.datalab.storage as storage
from io import BytesIO
mybucket = storage.Bucket('machinelearning763finalproject')
data_csv = mybucket.object('paysim.csv')
uri= data_csv.uri 
%gcs read --object $uri --variable data_csv
#Loading the dataset
dataset = pd.read_csv(BytesIO(data_csv))

In [4]:
dataset['relativehour'] = (dataset.step % 24).astype(str)
dataset['origErrorAmt'] = dataset.newbalanceOrig + dataset.amount - dataset.oldbalanceOrg
dataset['destErrorAmt'] = dataset.oldbalanceDest + dataset.amount - dataset.newbalanceDest
dataset['origTansPerc'] = ((dataset.amount / dataset.oldbalanceOrg) > 0.99)*1

X_all = dataset.iloc[:, [0,1,11,12,13,14]]
#X_all['relativehour'] = X_all['relativehour'].astype(str)
X_all = pd.get_dummies(X_all, drop_first=True)
y_all = dataset.loc[:,'isFraud']

In [5]:
datasetTrim = dataset.loc[(dataset.type == 'TRANSFER') | (dataset.type == 'CASH_OUT')]
#dataset['type'] = dataset.type.map({'TRANSFER': 0, 'CASH_OUT': 1})
X_trim = datasetTrim.iloc[:, [0,1,11,12,13,14]]
X_trim = pd.get_dummies(X_trim, drop_first=True)
y_trim = datasetTrim.loc[:,'isFraud']

In [ ]:
sc_X = StandardScaler()
#X_train = sc_X.fit_transform(X_train)
#X_test = sc_X.transform(X_test)
X_all = sc_X.fit_transform(X_all)
X_trim = sc_X.fit_transform(X_trim)

/usr/local/envs/py3env/lib/python3.5/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


In [ ]:
smt=SMOTE()
X_all, y_all = smt.fit_sample(X_all, y_all)
X_trim, y_trim = smt.fit_sample(X_trim, y_trim)